In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import torch.hub 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam



In [17]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=False)
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, 3 )

Using cache found in C:\Users\nisha/.cache\torch\hub\pytorch_vision_v0.10.0


In [18]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [19]:
dataset = datasets.ImageFolder(r"C:\Users\nisha\COMP6721\signs_train\asl_alphabet_train\asl_alphabet_train", transform=preprocess)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [20]:
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [21]:
len(dataset.classes)

3

In [22]:
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 20
total_steps = len(train_dataloader)

t1 = time.time()

for epoch in range(num_epochs):
    for i, data in enumerate(train_dataloader):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/20], Step [50/141], Loss: 0.1660, Accuracy: 92.19%
Epoch [1/20], Step [100/141], Loss: 0.0256, Accuracy: 100.00%
Epoch [2/20], Step [50/141], Loss: 0.0069, Accuracy: 100.00%
Epoch [2/20], Step [100/141], Loss: 0.0005, Accuracy: 100.00%
Epoch [3/20], Step [50/141], Loss: 0.0003, Accuracy: 100.00%
Epoch [3/20], Step [100/141], Loss: 0.0002, Accuracy: 100.00%
Epoch [4/20], Step [50/141], Loss: 0.0002, Accuracy: 100.00%
Epoch [4/20], Step [100/141], Loss: 0.0276, Accuracy: 98.44%
Epoch [5/20], Step [50/141], Loss: 0.0005, Accuracy: 100.00%
Epoch [5/20], Step [100/141], Loss: 0.3840, Accuracy: 93.75%
Epoch [6/20], Step [50/141], Loss: 0.0011, Accuracy: 100.00%
Epoch [6/20], Step [100/141], Loss: 0.0003, Accuracy: 100.00%
Epoch [7/20], Step [50/141], Loss: 0.0006, Accuracy: 100.00%
Epoch [7/20], Step [100/141], Loss: 0.0000, Accuracy: 100.00%
Epoch [8/20], Step [50/141], Loss: 0.0001, Accuracy: 100.00%
Epoch [8/20], Step [100/141], Loss: 0.0009, Accuracy: 100.00%
Epoc

In [10]:
model.eval() 

with torch.no_grad(): 
    correct = 0
    total = 0
    for data in test_dataloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))

Test Accuracy of the model on the 9000 test images: 98.7 %
